## Load Files Demographics

In [2]:
import pandas as pd

In [59]:
demo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport')
bp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT', format='xport')
body = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BMX_L.XPT', format='xport')
chol_total = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/TCHOL_L.XPT', format='xport')
exam = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT', format='xport')
glycohemo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/GHB_L.XPT', format='xport')
crp = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/HSCRP_L.XPT', format='xport')
dm = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DIQ_L.XPT', format='xport')
sedentary = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/PAQ_L.XPT', format='xport')
sr_wt = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/WHQ_L.XPT', format='xport')
kidney = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/KIQ_U_L.XPT', format='xport')

###### ***Question 1: "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"***

In [21]:
## Modify marriage and education data to only use values that we are concerned with

marr = demo.loc[demo['DMDMARTZ'] < 4]
edu = demo.loc[demo['DMDEDUC2'] < 6]

## Replace code values with description
mod_marr = marr['DMDMARTZ'].replace({1: 'Married', 2: 'Single', 3: 'Single'})
mod_edu = edu['DMDEDUC2'].replace({1: 'Below college', 2: 'Below college', 3: 'Below college', 4: 'College or above', 5: 'College or above'})

mod_marr.sample(10)
mod_edu.sample(10)

## create contingency table
cont_table = pd.crosstab(mod_marr, mod_edu)
print(cont_table)

DMDEDUC2  Below college  College or above
DMDMARTZ                                 
Married            1352              2782
Single             1431              2207


In [30]:
from scipy.stats import chi2_contingency

stat, p_value, dof, expected = chi2_contingency(cont_table)

print("Chi-square statistic:", {stat})
print("P-value:", {p_value})
print("Expected frequencies:", expected)

if p_value > 0.05:
    print("No significant association between marital status and education level.")
else:
    print("Significant association between marital status and education level.")

Chi-square statistic: {36.72203920845173}
P-value: {1.3623057407211713e-09}
Expected frequencies: [[1480.30391148 2653.69608852]
 [1302.69608852 2335.30391148]]
Significant association between marital status and education level.


###### ***Question 2: "Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?"***

In [39]:
from scipy.stats import ttest_ind

# Filter and prepare marital status data
demo_filtered = demo.loc[demo['DMDMARTZ'] < 4]
demo_filtered['Marital_Status'] = demo_filtered['DMDMARTZ'].replace({1: 'Married', 2: 'Single', 3: 'Single'})


# Merge demo and sedentary data on a common identifier
merged_data = pd.merge(demo_filtered[['SEQN', 'Marital_Status']], sedentary[['SEQN', 'PAD680']], on='SEQN')

# Filter for valid sedentary behavior times
merged_data = merged_data[merged_data['PAD680'] <= 1380]

# Separate sedentary behavior times by marital status
married_sedentary = merged_data.loc[merged_data['Marital_Status'] == 'Married', 'PAD680']
not_married_sedentary = merged_data.loc[merged_data['Marital_Status'] == 'Single', 'PAD680']

# Perform independent t-test
stat, p_value = ttest_ind(married_sedentary, not_married_sedentary)
print(f"Independent t-Test Statistic: {stat}, p-value: {p_value}")

if p_value > 0.05:
    print("No significant difference in sedentary behavior time between married and not-married individuals.")
else:
    print("Significant difference in sedentary behavior time between married and not-married individuals.")

Independent t-Test Statistic: -3.8699896847970154, p-value: 0.00010973792037934772
Significant difference in sedentary behavior time between married and not-married individuals.


<ipython-input-39-a27f8fffcff2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_filtered['Marital_Status'] = demo_filtered['DMDMARTZ'].replace({1: 'Married', 2: 'Single', 3: 'Single'})


###### ***Question 3: "How do age and marital status affect systolic blood pressure?""***

In [42]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Select the relevant columns
age_data = demo['RIDAGEYR']  # Age column
marital_status = demo['DMDMARTZ'].replace({1: 'Married', 2: 'Single', 3: 'Single'})  # Cleaned marital status data
systolic_bp = exam['BPXOSY3'].dropna()  # Systolic blood pressure column with missing values dropped

# Combine selected data into a single DataFrame
anova_data = pd.concat([age_data, marital_status, systolic_bp], axis=1)
anova_data.columns = ['Age', 'Marital_Status', 'Systolic_BP']  # Renaming for clarity

# Fit an ANOVA model to analyze the effect of age and marital status on systolic blood pressure
model = ols('Systolic_BP ~ Age + Marital_Status', data=anova_data).fit()
anova_results = sm.stats.anova_lm(model, typ=2)

# Display ANOVA table and model summary
print(anova_results)
print(model.summary())

# Interpretation based on p-value from ANOVA
p_value = anova_results['PR(>F)'][0]  # Extract p-value for 'Age' (you may need to adjust index for specific p-value)
if p_value > 0.05:
    print("No significant effect of age and marital status on systolic blood pressure.")
else:
    print("Significant effect of age and marital status on systolic blood pressure.")


                      sum_sq      df         F    PR(>F)
Marital_Status  3.149985e+02     3.0  0.299184  0.826016
Age             9.214067e+01     1.0  0.262544  0.608401
Residual        1.711948e+06  4878.0       NaN       NaN
                            OLS Regression Results                            
Dep. Variable:            Systolic_BP   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.2872
Date:                Thu, 07 Nov 2024   Prob (F-statistic):              0.886
Time:                        06:07:15   Log-Likelihood:                -21235.
No. Observations:                4883   AIC:                         4.248e+04
Df Residuals:                    4878   BIC:                         4.251e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                        

<ipython-input-42-4e6f9dd709be>:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_value = anova_results['PR(>F)'][0]  # Extract p-value for 'Age' (you may need to adjust index for specific p-value)


###### ***Question 4: "Is there a correlation between self-reported weight and minutes of sedentary behavior?""***

In [58]:
from scipy.stats import pearsonr
from scipy.stats import spearmanr

weight_data = sr_wt.loc[sr_wt['WHD020'] <=530]
sedentary_data = sedentary['PAD680'].dropna()

# Ensure both variables have the same length (aligning by index)
aligned_data = pd.concat([weight_data, sedentary_data], axis=1, join='inner')
aligned_data.columns = ['Weight', 'Sedentary_Time']

# Calculate Pearson correlation
correlation, p_value = pearsonr(aligned_data['Weight'], aligned_data['Sedentary_Time'])

# Display the results
print(f"Pearson correlation coefficient: {correlation}")
print(f"P-value: {p_value}")

# Interpretation of the correlation
if p_value > 0.05:
    print("There is no significant correlation between self-reported weight and minutes of sedentary behavior.")
else:
    print("There is a significant correlation between self-reported weight and minutes of sedentary behavior.")


ValueError: Length mismatch: Expected axis has 6 elements, new values have 2 elements

###### ***Question 5 (Creative Analysis): Develop your own unique question using at least one of the variables listed above. Ensure that your question can be answered using one of the following tests: chi-square, t-test, ANOVA, or correlation. Clearly state your question, explain why you chose the test, and document your findings.***

- Is there a relationship between age and the presence of failing kidneys?

In [61]:
import pandas as pd
from scipy.stats import ttest_ind

# Assuming the necessary data is loaded into 'demo' DataFrame

# Step 1: Recode weak/failing kidneys status (KIQ022)
demo['Kidney_Status'] = demo['KIQ022'].apply(
    lambda x: 'Failing Kidneys' if x == 1 else 'No Failing Kidneys'
)

# Step 2: Clean age data (RIDAGEYR) and remove missing values
demo_clean = demo[['RIDAGEYR', 'Kidney_Status']].dropna(subset=['RIDAGEYR'])

# Step 3: Group by kidney status
failing_kidneys = demo_clean[demo_clean['Kidney_Status'] == 'Failing Kidneys']['RIDAGEYR']
no_failing_kidneys = demo_clean[demo_clean['Kidney_Status'] == 'No Failing Kidneys']['RIDAGEYR']

# Step 4: Perform an independent t-test
stat, p_value = ttest_ind(failing_kidneys, no_failing_kidneys)

# Step 5: Display results
print(f"T-Test Statistic: {stat}")
print(f"P-value: {p_value}")

# Step 6: Interpretation
if p_value > 0.05:
    print("There is no significant difference in age between individuals with and without failing kidneys.")
else:
    print("There is a significant difference in age between individuals with and without failing kidneys.")



KeyError: 'KIQ022'